This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [82]:
def basicPrint(model, **kwargs):
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': 2,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        'metricsFilter': {'ndcg', 'div@100', 'topic-div@100', 'swjacc-div@100', 'jacc-div@100'}, # None, {'ndcg', 'ndcg@10'}
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

# Misc

In [ ]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

In [ ]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

In [77]:
# Removing combin models with maxUsers is not None:
if False:
    twinewsRankings = getTwinewsRankings()
    ids = twinewsRankings.keys()
    bp(ids)
    for currentId in ids:
        if 'combin' in currentId:
            meta = twinewsRankings.getMeta(currentId)
            if meta['maxUsers'] is not None:
                del twinewsRankings[currentId]

# Model lists per split version

In [67]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [68]:
printModelList(2)

BM25_yf DSSM_yf bert bm25 combin dbert-base dbert-ft doc2vec ideal infersent lda nmf random sent2vec stylo tfidf usent word2vec worst (19)


In [69]:
printModelList(1)

bert bm25 combin dbert-base dbert-ft doc2vec ideal infersent lda nmf random sent2vec stylo tfidf usent word2vec worst (17)


# Reference models

In [84]:
splitVersion = 2
df = printReport(model='ideal', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='random', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='worst', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')

,id,maxUsers,model,splitVersion,div@100,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100,style-div@100,topic-div@100
0,ideal-5ac2d,None,ideal,2,0.97696,1,1,1,1,1,0.56539,0.07227,0.61229,0.89322


These values are common to all rows (10):

	- splitVersion: 2
	- maxUsers: None
	- model: random


,id,seed,div@100,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100,style-div@100,topic-div@100
9,random-f03b8,4,0.97668,0.0109,0.01815,0.22153,0.00019,0.05189,0.0003,0.01156,0.61622,0.8943
7,random-4d05f,8,0.97674,0.0096,0.01258,0.21667,0.00282,0.0034,0.00442,0.00137,0.61873,0.89431
4,random-6fae4,3,0.97635,0.00979,0.00786,0.21645,0,0.00067,0,0.00042,0.61758,0.89302
3,random-04de9,2,0.97658,0.00869,0.0086,0.21099,0,0.00098,0,0.00048,0.61526,0.89369
8,random-a9655,1,0.97633,0.0083,0.00934,0.21023,0.00027,0.02439,0.00041,0.00688,0.61847,0.89294
0,random-21e0b,6,0.97683,0.00871,0.02293,0.20917,0.01098,0.00966,0.01582,0.00261,0.61798,0.89466
1,random-d87ed,9,0.9763,0.00773,0.00606,0.20841,0,0.00015,0,0.00012,0.62007,0.89246
2,random-a1ece,7,0.97658,0.00828,0.01206,0.20738,0,0.0069,0,0.00195,0.61901,0.89424
6,random-093c0,0,0.97669,0.00783,0.01652,0.20427,0,0.01671,0,0.00396,0.61644,0.89466
5,random-e27ab,5,0.97664,0.00738,0.00755,0.20295,0.0002,0.0172,0.00028,0.00583,0.61735,0.89412


,id,maxUsers,model,splitVersion,div@100,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100,style-div@100,topic-div@100
0,worst-f3f99,None,worst,2,0.9777,0.00413,0.001,0.18191,0,0,0,0,0.61269,0.89509


# Ad-hoc models

In [83]:
basicPrint('bm25')

These values are common to all rows (26):

	- splitVersion: 2
	- maxUsers: None
	- epsilon: 0.25
	- model: bm25


,id,b,doLemmatization,k1,lowercase,maxDF,minDF,div@100,jacc-div@100,ndcg,topic-div@100
17,bm25-1eb2a,1.2,False,2.4,False,300,0.0005,0.95316,nan,0.55949,0.80043
15,bm25-e8059,1.2,False,2.6,False,300,0.0005,0.95398,nan,0.5592,0.80114
5,bm25-efa94,0.9,False,2.4,False,300,0.0005,0.94662,nan,0.54458,0.79973
20,bm25-f3052,0.9,False,2.2,False,300,0.0005,0.94617,nan,0.54253,0.79976
25,bm25-5481b,0.9,False,2,False,300,0.0005,0.94567,nan,0.5398,0.79989
18,bm25-9e342,1.4,False,2.4,False,300,0.0005,0.96133,nan,0.53963,0.81175
7,bm25-87073,0.75,False,2.4,False,300,0.0005,0.94423,nan,0.52998,0.80149
2,bm25-f55be,0.75,False,2.2,False,300,0.0005,0.9439,nan,0.52757,0.80164
3,bm25-2c4b0,1.4,False,2.6,False,300,0.0005,0.96368,nan,0.52735,0.81719
19,bm25-a91ce,0.75,False,2,False,300,0.0005,0.94355,nan,0.52465,0.80184


In [27]:
basicPrint('BM25_yf', noSubsampling=False)

Found key difference: {'empiricalParameter'}
These values are common to all rows (2):

	- maxDF: 300
	- lowercase: False
	- maxDocuments: 10000
	- splitVersion: 2
	- maxUsers: 30
	- doLemmatization: True
	- minDF: 0.0005
	- useExtraNews: False
	- model: BM25_yf
	- epsilon: 0.25
	- implementation: BM25
	- b: 0.75


,id,empiricalParameter,k1,ndcg
1,BM25_yf-78651,N/A,1.8,0.5355
0,BM25_yf-42d08,True,1.2,0.52538


# Models that use vector representations

In [26]:
basicPrint('lda')

These values are common to all rows (18):

	- ldaLearningMethod: online
	- maxDF: 300
	- lowercase: True
	- model: lda
	- splitVersion: 2
	- maxUsers: None
	- useExtraNews: False
	- ldaLearningDecay: 0.7


,id,distance,doLemmatization,historyRef,implementation,ldaLearningOffset,maxDocuments,maxIter,minDF,nbTopics,useTFIDF,div@100,ndcg
4,lda-d4f1d,cosine,False,30,gensim-lda,1,300000,60,0.0005,100,False,nan,0.41897
14,lda-b123f,cosine,False,30,gensim-lda,1,100000,120,0.0005,100,False,nan,0.41415
17,lda-d1220,cosine,False,0.3,gensim-lda,1,100000,60,0.0005,100,False,nan,0.38648
5,lda-af807,cosine,False,30,gensim-lda,1,10000,60,0.0005,100,False,0.96109,0.37941
1,lda-759de,cosine,False,0.6,gensim-lda,1,10000,60,0.0005,100,False,nan,0.37484
12,lda-64d0c,cosine,False,1.0,gensim-lda,1,10000,60,0.0005,100,False,nan,0.37478
6,lda-2fa8e,euclidean,False,0.3,gensim-lda,1,10000,60,0.0005,100,False,0.96097,0.36168
0,lda-b849e,cosine,False,0.3,gensim-lda,1,10000,60,0.0005,300,False,nan,0.35651
2,lda-7786b,cosine,True,0.3,gensim-lda,1,10000,60,0.0005,100,False,nan,0.35505
9,lda-d3b82,cosine,False,0.3,gensim-lda,1,10000,60,0.0005,100,False,nan,0.35335


In [ ]:
basicPrint('nmf')

In [45]:
basicPrint('tfidf')

Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
These values are common to all rows (22):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: tfidf
	- sublinearTF: True


,id,dimensions,doLemmatization,historyRef,lowercase,maxDF,maxIter,minDF,div@100,ndcg
1,tfidf-be107,nan,False,0.4,False,300,N/A,0.0005,nan,0.54867
19,tfidf-6c83e,nan,False,0.35,False,300,N/A,0.0005,nan,0.54863
0,tfidf-01001,nan,False,0.3,False,300,N/A,0.0005,0.94452,0.5484
2,tfidf-7c894,nan,False,0.45,False,300,N/A,0.0005,nan,0.54769
6,tfidf-f57a2,nan,False,0.5,False,300,N/A,0.0005,nan,0.54631
14,tfidf-11f90,nan,False,0.55,False,300,N/A,0.0005,0.94251,0.54509
15,tfidf-9bb25,nan,False,0.6,False,300,N/A,0.0005,0.94224,0.54388
10,tfidf-e85ec,nan,False,0.2,False,300,N/A,0.0005,nan,0.54294
8,tfidf-6056f,nan,False,30,False,300,N/A,0.0005,0.94118,0.53276
5,tfidf-487b6,3000,False,0.3,False,300,10,0.0005,0.94558,0.53182


In [56]:
basicPrint('dbert-ft')

These values are common to all rows (16):

	- splitVersion: 2
	- maxUsers: None
	- model: dbert-ft


,id,distance,historyRef,div@100,ndcg
3,dbert-ft-d1b5f,cosine,0.35,0.96112,0.49254
15,dbert-ft-6c7f3,cosine,0.4,0.96104,0.49243
10,dbert-ft-71ce1,cosine,0.3,0.96118,0.49234
8,dbert-ft-79517,cosine,0.45,0.96101,0.49147
12,dbert-ft-2529b,cosine,0.6,nan,0.48681
9,dbert-ft-9cd08,cosine,10,0.96086,0.48658
7,dbert-ft-d66a1,cosine,0.2,0.96156,0.48567
1,dbert-ft-833d3,cosine,3,0.96147,0.485
0,dbert-ft-a259b,cosine,20,nan,0.4785
6,dbert-ft-c3c19,cosine,30,0.9609,0.47426


In [57]:
basicPrint('dbert-base')

These values are common to all rows (8):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: dbert-base


,id,historyRef,div@100,ndcg
0,dbert-base-aafd1,0.3,nan,0.41968
6,dbert-base-948f8,0.35,nan,0.41866
5,dbert-base-0eed3,0.4,0.95648,0.41691
1,dbert-base-68550,0.5,0.95631,0.41324
4,dbert-base-3f966,10,0.95664,0.41003
2,dbert-base-019e2,0.6,0.95627,0.40943
3,dbert-base-72c66,0.1,0.96002,0.40695
7,dbert-base-b7bd7,30,0.95753,0.38931


In [ ]:
basicPrint('stylo')

In [15]:
basicPrint('infersent')

These values are common to all rows (9):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: infersent


,id,historyRef,ndcg
8,infersent-02118,0.3,0.41838
7,infersent-d3c3c,0.35,0.41774
0,infersent-9c7be,0.4,0.41677
5,infersent-0e427,0.45,0.41542
1,infersent-92160,0.55,0.41216
2,infersent-590a1,10,0.41106
4,infersent-a88c4,0.6,0.41002
6,infersent-b167b,30,0.39663
3,infersent-2c725,1.0,0.39237


In [ ]:
basicPrint('sent2vec')

In [40]:
basicPrint('word2vec')

These values are common to all rows (5):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: word2vec


,id,historyRef,div@100,ndcg
1,word2vec-4a2c5,0.35,nan,0.22383
4,word2vec-f0f25,0.3,0.97663,0.22337
0,word2vec-d2dd2,0.4,nan,0.22324
3,word2vec-ad554,1.0,0.97668,0.22307
2,word2vec-6a91b,30,nan,0.22279


In [25]:
basicPrint('doc2vec')

These values are common to all rows (10):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: doc2vec


,id,historyRef,div@100,ndcg
9,doc2vec-19ca3,0.5,nan,0.53493
3,doc2vec-fd25d,0.55,nan,0.5346
7,doc2vec-3aab3,0.6,nan,0.53423
1,doc2vec-659d6,0.4,0.95457,0.5339
0,doc2vec-7c41f,10,0.95436,0.53379
4,doc2vec-e2911,0.35,0.95476,0.53249
2,doc2vec-b6a4f,0.3,nan,0.53122
6,doc2vec-1ee6f,30,0.95436,0.52738
5,doc2vec-6c13f,1.0,nan,0.52528
8,doc2vec-c01b6,1,nan,0.49074


In [61]:
basicPrint('usent')

These values are common to all rows (11):

	- distance: cosine
	- splitVersion: 2
	- maxUsers: None
	- model: usent


,id,historyRef,div@100,ndcg
0,usent-b1150,0.6,0.96224,0.4687
5,usent-96ae6,0.55,0.96221,0.46854
10,usent-545cd,0.5,0.96219,0.46823
4,usent-1f67c,0.45,0.96219,0.46752
3,usent-9e577,10,0.96228,0.46741
6,usent-9a8cc,0.4,0.9622,0.46686
2,usent-24fe1,30,0.96252,0.4659
9,usent-90a9a,0.35,0.96225,0.46542
1,usent-617a7,1.0,0.96262,0.46507
8,usent-db26e,0.3,0.9623,0.46422


In [ ]:
basicPrint('bert')

# Combinations of models

In [80]:
# Expected 156 rows:
basicPrint('combin')

These values are common to all rows (156):

	- betas: ['LOG', 'LOG']
	- splitVersion: 2
	- maxUsers: None
	- alphas: [0.5, 0.5]
	- weights: [0.5, 0.5]
	- model: combin


,id,models,rankAsScore,div@100,ndcg
140,combin-cf91b,"['bm25-1eb2a', 'dbert-ft-d1b5f']","[False, False]",0.95478,0.58772
118,combin-c56e2,"['dbert-ft-d1b5f', 'tfidf-be107']","[False, False]",0.94992,0.58034
79,combin-f3aa7,"['bm25-1eb2a', 'doc2vec-19ca3']","[False, False]",0.95165,0.57265
47,combin-e7de3,"['bm25-1eb2a', 'tfidf-be107']","[False, False]",0.94714,0.56678
8,combin-84cca,"['bm25-1eb2a', 'tfidf-be107']","[True, True]",0.94769,0.56655
27,combin-6b411,"['dbert-ft-d1b5f', 'doc2vec-19ca3']","[False, False]",0.95511,0.56448
37,combin-a9c56,"['bm25-1eb2a', 'doc2vec-19ca3']","[True, True]",0.95123,0.56408
71,combin-01199,"['doc2vec-19ca3', 'tfidf-be107']","[False, False]",0.94715,0.56279
113,combin-1e53b,"['bm25-1eb2a', 'dbert-base-aafd1']","[False, False]",0.95162,0.5598
125,combin-7d8fe,"['bm25-1eb2a', 'word2vec-4a2c5']","[False, False]",0.95316,0.55949


# Deep Learning models

In [70]:
basicPrint('DSSM_yf', noSubsampling=False)

,id,batch_size,char_embedding_size,class_size,doLemmatization,epochs,historyRef,keep_prob,learning_rate,lowercase,maxDF,maxDocuments,maxUsers,minDF,model,seq_length,similarity,splitVersion,useExtraNews,vocab_size,div@100,ndcg
0,DSSM_yf-d9604,100,100,2,False,10,0.3,0.7,0.0005,False,300,100,10,0.002,DSSM_yf,7427,cosine,2,0,7901,0.99126,0.21601


# Reports on the test set (split version 1)

In [36]:
splitVersion = 1

In [47]:
report = printReport(splitVersion=splitVersion, model='ideal')
report = printReport(splitVersion=splitVersion, model='random')
report = printReport(splitVersion=splitVersion, model='worst')

,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,ideal-5eb2b,None,ideal,1,1,1,1,1,1,0.67484,0.1063


These values are common to all rows (10):

	- splitVersion: 1
	- maxUsers: None
	- model: random


,id,seed,div@100,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
1,random-03419,8,0.97687,0.01278,0.01932,0.24411,0.0076,0.00772,0.01179,0.00342
0,random-15c7e,4,nan,0.0146,0.02233,0.25101,0.00103,0.0575,0.00156,0.0164
2,random-8373e,9,nan,0.0101,0.0066,0.23379,1e-05,0.00072,1e-05,0.00064
3,random-a039c,3,nan,0.01305,0.0088,0.24404,1e-05,0.00206,2e-05,0.00148
4,random-fbe0a,5,nan,0.011,0.0119,0.2342,0.00143,0.02565,0.00198,0.01001
5,random-bf97b,0,nan,0.01139,0.023,0.23543,5e-05,0.02322,7e-05,0.00656
6,random-ab01e,6,nan,0.01294,0.0388,0.24158,0.02029,0.01828,0.02917,0.00596
7,random-35ee6,7,nan,0.01209,0.01815,0.2383,0,0.01147,0,0.00383
8,random-8e615,1,nan,0.01166,0.01222,0.23957,0.00127,0.03055,0.00192,0.01023
9,random-8217a,2,nan,0.01218,0.0113,0.24015,0.00013,0.00276,0.00012,0.00159


,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,worst-559a0,None,worst,1,0.00587,0.001,0.20599,0,0,0,0


In [60]:
report = printReport\
(
    splitVersion=splitVersion,
    excludedModels={'combin', 'ideal', 'worst', 'random'},
    # metricsFilter={'ndcg'},
    # metricsFilter={'ndcg', 'div@100'},
    sortBy='ndcg',
    noSubsampling=True,
    onlyFields={'id', 'model'}
)

,id,model,div@100,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100,style-div@100,topic-div@100
13,bm25-933f7,bm25,0.95301,0.36316,0.61723,0.60819,0.44273,0.53604,0.31301,0.07674,0.56055,0.79632
12,tfidf-4a1dc,tfidf,0.94389,0.34771,0.60442,0.59695,0.42603,0.52076,0.29923,0.07479,0.5462,0.80162
11,doc2vec-e013a,doc2vec,0.95414,0.3232,0.58795,0.58202,0.40184,0.50388,0.28369,0.07424,0.5541,0.81484
10,tfidf-4b89a,tfidf,0.95211,0.30134,0.55199,0.56138,0.37432,0.47773,0.26487,0.0715,0.56383,0.82701
5,nmf-6078e,nmf,0.95918,0.26979,0.51627,0.54098,0.33832,0.4599,0.24526,0.07211,0.56764,0.76415
2,dbert-ft-7847a,dbert-ft,0.96076,0.27155,0.53054,0.53664,0.3472,0.44411,0.24932,0.06864,0.46444,0.84312
9,usent-64ec0,usent,0.96142,0.24835,0.49151,0.5172,0.31744,0.41832,0.22694,0.06454,0.56427,0.8148
7,lda-82272,lda,0.96183,0.19536,0.40872,0.47486,0.24493,0.37615,0.18029,0.06437,0.57525,0.78475
8,dbert-base-d092a,dbert-base,0.95702,0.18273,0.43752,0.46352,0.25297,0.34227,0.17994,0.05452,0.54249,0.82539
3,infersent-77ec7,infersent,0.95077,0.18162,0.42212,0.46136,0.246,0.34086,0.17558,0.05449,0.54746,0.8025


In [81]:
# Expected 156 rows:
report = printReport\
(
    splitVersion=splitVersion,
    model='combin',
    # metricsFilter={'ndcg'},
    # metricsFilter={'ndcg', 'div@100'},
    # metricsFilter={'ndcg', 'topic-div@100'},
    sortBy='ndcg',
    noSubsampling=True,
)

These values are common to all rows (156):

	- betas: ['LOG', 'LOG']
	- splitVersion: 1
	- maxUsers: None
	- alphas: [0.5, 0.5]
	- weights: [0.5, 0.5]
	- model: combin
